In [2]:
# !pip install pandas
# !pip install pinecone
# !pip unstall openai
# !pip install python-dotenv
# !pip install scikit-learn

In [3]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
import tiktoken
load_dotenv()

True

In [4]:
API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = f"https://api.openai.com/v1"
MODEL_NAME = os.getenv("MODEL_NAME")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL")

# API_KEY = os.getenv("RUNPOD_TOKEN")
# BASE_URL = os.getenv("RUNPOD_CHATBOT_URL")
# model_name = os.getenv("MODEL_NAME")

print(MODEL_NAME)
# print(API_KEY)

gpt-3.5-turbo-0125


In [5]:
def get_chatbot_response(client, model_name, messages,temperature=0):
    # Initialize the tokenizer for the model
    tokenizer = tiktoken.encoding_for_model(model_name)

    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    # Count tokens sent
    tokens_sent = sum(len(tokenizer.encode(message["content"])) for message in messages)

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content

    # Count tokens received
    tokens_received = len(tokenizer.encode(response))
    print("*" * 36)
    # print(f"Tokens:")
    print(f"Tokens sent: {tokens_sent}, Tokens received: {tokens_received}")  # Print token counts
    print("*" * 36)

    return response

In [6]:
import json

def print_pretty_json(response):
    """
    Prints the JSON response in a pretty format.

    Parameters:
    response (str): The JSON string to be printed.
    """
    try:
        json_response = json.loads(response)  # Load the JSON string into a Python dictionary
        print("Response: ")
        print("*" * 36)
        print(json.dumps(json_response, indent=4))  # Pretty print the JSON
        print("*" * 36)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:")
        print(e)
        print("Response content:")
        print(response)


# Example usage
# response = '{"name": "John", "age": 30, "city": "New York"}'
# print_pretty_json(response)

In [7]:
client = OpenAI(
            api_key=API_KEY,
            base_url=BASE_URL,
        )
# model_name = os.getenv("MODEL_NAME")

# Get LLM response

In [9]:
messages = [{'role':'user', 'content': "What's the capital of usa?"}]
response = get_chatbot_response(client, MODEL_NAME, messages)
print(response)

************************************
Tokens sent: 7, Tokens received: 14
************************************
The capital of the United States of America is Washington, D.C.


# Prompt engineering

## Structred output

In [12]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    country: the country that you will get the capital of 
    capital: the capital of the country stated
}
]
"""
messages = [{'role': 'system', 'content': system_prompt}]
messages.append({'role': 'user', 'content': "What's the capital of Italy?"})

In [13]:
messages[0]

{'role': 'system',
 'content': '\nYou are a helpful assistant that answer questions about capitals of countries.\n\nYour output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:\n[\n{\n    country: the country that you will get the capital of \n    capital: the capital of the country stated\n}\n]\n'}

In [14]:
response = get_chatbot_response(client, MODEL_NAME, messages)
print_pretty_json(response)

************************************
Tokens sent: 77, Tokens received: 21
************************************
Response: 
************************************
[
    {
        "country": "Italy",
        "capital": "Rome"
    }
]
************************************


## input structuring

In [16]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})

In [17]:
messages

[{'role': 'system',
  'content': '\nYou are a helpful assistant that answer questions about capitals of countries.\n\nYour output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:\n[\n{\n    country: the country that you will get the capital of \n    capital: the capital of the country stated\n}\n]\n'},
 {'role': 'user',
  'content': '\nGet me the capitals of the following countries:\n```\n1. Italy\n2. France\n3. Germany\n``\n'}]

In [18]:
response = get_chatbot_response(client, MODEL_NAME, messages)
print_pretty_json(response)

************************************
Tokens sent: 95, Tokens received: 58
************************************
Response: 
************************************
[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "France",
        "capital": "Paris"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    }
]
************************************


## Give the model time to think (Chain of thought)

> https://arxiv.org/pdf/2205.11916

In [20]:
# equation = "1+3"
equation = "259/2*8654+91072*33-12971"

user_prompt = """
Calculate the result of this equation: {equation}
Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{{
    "result": The final number resulted from calculating the equation above
}}
"""

user_formatted_prompt = user_prompt.format(equation=equation)

messages = [{'role': 'user', 'content': user_formatted_prompt}]
response = get_chatbot_response(client, MODEL_NAME, messages)
print_pretty_json(response)

************************************
Tokens sent: 68, Tokens received: 11
************************************
Response: 
************************************
{
    "result": 3019935
}
************************************


## Improved Prompt

In [22]:
equation = "259/2*8654+91072*33-12971"

user_prompt = """
Calculate the result of this equation: {equation}

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. 
    You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}}
"""

user_formatted_prompt = user_prompt.format(equation=equation)

messages = [{'role': 'user', 'content': user_formatted_prompt}]
response = get_chatbot_response(client, MODEL_NAME, messages)
print_pretty_json(response)

************************************
Tokens sent: 116, Tokens received: 89
************************************
Response: 
************************************
{
    "steps": [
        "259 / 2 = 129.5",
        "129.5 * 8654 = 1120733",
        "91072 * 33 = 3008496",
        "1120733 + 3008496 = 4129229",
        "4129229 - 12971 = 4116258"
    ],
    "result": 4116258
}
************************************


In [23]:
4130258 - 3019935

1110323

# RAG - Retrieval Augmented Generation

#### Asking about a subject that the LLM does not know anything about

In [26]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role': 'user', 'content': user_prompt}]
response = get_chatbot_response(client, MODEL_NAME, messages)
print_pretty_json(response)

************************************
Tokens sent: 9, Tokens received: 55
************************************
Error decoding JSON:
Expecting value: line 1 column 1 (char 0)
Response content:
As of now, there is no information available about an iPhone 16 as Apple has not released any details or announcements about a new iPhone model beyond the iPhone 13. It is best to stay updated with Apple's official announcements and news for any updates on future iPhone models.


#### Giving Context to the unknown subject

In [28]:
question = "What chip is used in iphone 16?"

In [29]:
system_prompt = """
You are a helpful assistant that answer questions.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    Question: The question asked
    Answer: the answer of the question asked
}
]
"""

In [30]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. 
It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. 
The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, 
faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. 
The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined 
audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery 
of any iPhone 9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. 
The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [31]:
user_prompt = f"""
Context:
{iphone_16}

Question:
{question}
"""

user_formatted_prompt = user_prompt.format(iphone_16=iphone_16, question=question)
print(user_formatted_prompt)


Context:

The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. 
It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. 
The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, 
faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. 
The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses ma

In [32]:
messages = [{'role': 'system', 'content': system_prompt}]
messages.append({'role': 'user', 'content': user_formatted_prompt})
response = get_chatbot_response(client, MODEL_NAME, messages)
print_pretty_json(response)
# print(response)

************************************
Tokens sent: 412, Tokens received: 37
************************************
Response: 
************************************
[
    {
        "Question": "What chip is used in iphone 16?",
        "Answer": "The iPhone 16 is powered by the new A18 chip."
    }
]
************************************


#### Automatically extract context data from database

In [142]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. 
It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. 
This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. 
It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. 
The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes 
productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, 
charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [144]:
system_prompt = """
You are a helpful assistant that answer questions.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    Question: The question asked
    Answer: the answer of the question asked
}
]
"""

In [145]:
data = [iphone_16, samsung_s23]

In [146]:
question = """What's new in iphone 16?"""
user_prompt = question

## Create Embeddings

In [150]:
embedding_client = OpenAI(
        api_key=API_KEY,
        base_url=BASE_URL,
)

In [152]:
def get_embedding(embedding_client, embedding_model_name, text_input):
    output = embedding_client.embeddings.create(input=text_input, model=embedding_model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [153]:
user_prompt_embeddings = get_embedding(embedding_client, EMBEDDING_MODEL, user_prompt)

In [154]:
# user_prompt_embeddings

In [155]:
user_prompt_embeddings = user_prompt_embeddings[0]

In [157]:
# user_prompt_embeddings

In [158]:
data_embeddings = [get_embedding(embedding_client, EMBEDDING_MODEL, x)[0] for x in data]

In [160]:
data_similaraty_scores = cosine_similarity([user_prompt_embeddings], data_embeddings)

In [161]:
data_similaraty_scores

array([[0.74983767, 0.40662091]])

In [165]:
closest_entry_index = data_similaraty_scores.argmax()
closest_entry_index

np.int64(0)

In [167]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. \nIt features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. \nThe iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, \nfaster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. \nThe Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses m

In [169]:
context = data[closest_entry_index]

In [175]:
user_prompt_with_data = f"""
Context: {context}

Question: {user_prompt}
"""

user_formatted_prompt_with_data = user_prompt_with_data.format(context=context, user_prompt=user_prompt)
print(user_formatted_prompt_with_data)


Context: 
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. 
It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. 
The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, 
faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. 
The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses ma

In [182]:
messages = [{'role': 'system', 'content': system_prompt}]
messages.append({'role': 'user', 'content': user_formatted_prompt_with_data})
response = get_chatbot_response(client, MODEL_NAME, messages)
print_pretty_json(response)
# print(response)

************************************
Tokens sent: 412, Tokens received: 101
************************************
Response: 
************************************
[
    {
        "Question": "What's new in iphone 16?",
        "Answer": "The iPhone 16 introduces several exciting updates, including larger displays, a more durable Ceramic Shield, the new A18 chip for improved performance, upgraded camera systems, advanced audio features like 'Audio Mix,' extended battery life, USB-C for faster charging and data transfer, and support for up to 2x faster video encoding in the Pro models. The starting prices remain consistent with previous generations."
    }
]
************************************
